In [ ]:
x1, x2, x3, x4 = var('x1 x2 x3 x4', domain='real')
y1, y2, y3, y4 = var('y1 y2 y3 y4', domain='real')
z1, z2, z3, z4 = var('z1 z2 z3 z4', domain='real')

p1 = vector((x1, y1, z1))
p2 = vector((x2, y2, z2))
p3 = vector((x3, y3, z3))
p4 = vector((x4, y4, z4))

e1 = p2-p1
e2 = p3-p1
e3 = p4-p1

n1 = e1.cross_product(e2)
n2 = e3.cross_product(e1)

In [ ]:
tan_theta = n1.cross_product(n2).norm() / (n1*n2)
theta = atan2(n2.cross_product(n1).norm(), n1*n2)

In [ ]:
psi = var('psi')
plot(theta.subs({x1:0,y1:0,z1:0, x2:1,y2:0,z2:0, x3:1,y3:1,z3:0, x4:0,y4:sin(psi),z4:cos(psi)}), (psi, 0, 10))

In [ ]:
e_bar = var('e_bar')
theta_bar = var('theta_bar')
h_bar = var('h_bar')

E = (theta - theta_bar) * (theta -theta_bar) * e_bar / h_bar

In [ ]:
from sympy.utilities.codegen import codegen
from sympy.simplify.cse_main import cse
from sympy.printing.pycode import pycode
from sympy.printing import ccode

vars = [x1, y1, z1, x2, y2, z2, x3, y3, z3, x4, y4, z4]
#indexed_hessian = ['d' + pycode(i) + pycode(j) for i in vars for j in vars]
#hessian = [E.diff(i).diff(j)._sympy_() for i in vars for j in vars]

In [ ]:
from sage.calculus.functions import jacobian
J = jacobian(E, vars)
H = jacobian(J, vars)

In [ ]:
[(c_name, c_code), (h_name, c_header)] = codegen(('J', J), 'C', 'flex_hessian')

In [ ]:
lhs, rhs = cse(hessian)
len(lhs)
len(rhs)

In [ ]:
codegen_list =[(pycode(v), t) for v, t in lhs] + list(zip(indexed_hessian, hessian))
[(c_name, c_code), (h_name, c_header)] = codegen(codegen_list, 'C', 'flex_hessian')

In [ ]:
o = open(c_name, 'w')
o.write(c_code)

In [ ]:
output = ""
for v, t in lhs:
    print(pycode(v) + ' = ' + pycode(t) )
for v, t in zip(indexed_hessian, rhs):
    print(v + ' = ' + pycode(t))

In [ ]:
output

In [ ]:
[(c_name, c_code), (h_name, c_header)] = codegen(("f", theta._sympy_()), "C", "test")
print(c_code)

In [ ]:
from sympy.utilities.autowrap import autowrap
func = autowrap(theta._sympy_())

In [ ]:
func(0,1,1,0, 0,0,1,0, 0,0,0,1)
func(0,1,1,0, 0,0,1,-1, 0,0,0,0)
func(0,1,1,0, 0,0,1,1, 0,0,0,1)

In [ ]:
float(pi/4)